In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder
import time
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("🚀 MODELO MELHORADO - CLICKPREDICT")
print("="*70)

# =====================================================
# CONFIGURAÇÕES OTIMIZADAS
# =====================================================
CONFIG_IMPROVED = {
    'MIN_SUPPORT': 0.0005,     # Reduzido para capturar mais padrões
    'MIN_CONFIDENCE': 0.2,     # Reduzido para mais regras
    'MIN_LIFT': 1.0,          # Reduzido para não filtrar demais
    'MAX_LEN': 2,             # Focar em pares diretos
    'TOP_RULES': 2000,        # Mais regras para melhor cobertura
}

# =====================================================
# FUNÇÃO MELHORADA DE PREPARAÇÃO
# =====================================================
def prepare_transactions_improved(df):
    """
    Preparação focada APENAS em rotas, sem metadados
    """
    print("\n📊 Preparando transações (versão melhorada)...")

    transactions = []

    # Processar por cliente
    for client_id in df['client_id'].unique():
        client_data = df[df['client_id'] == client_id]
        client_routes = []

        for _, row in client_data.iterrows():
            # APENAS rotas de ida
            if pd.notna(row['origin_departure']) and pd.notna(row['destination_departure']):
                route_dep = f"{row['origin_departure']}_to_{row['destination_departure']}"
                client_routes.append(route_dep)

            # APENAS rotas de volta (se existirem)
            if not row['no_return_flag']:
                if pd.notna(row['origin_return']) and pd.notna(row['destination_return']):
                    route_ret = f"{row['origin_return']}_to_{row['destination_return']}"
                    client_routes.append(route_ret)

        # Adicionar apenas se tiver pelo menos 2 rotas
        if len(client_routes) >= 2:
            transactions.append(client_routes)

    print(f"   • {len(transactions)} transações válidas criadas")
    print(f"   • Transações com 2+ rotas: {len([t for t in transactions if len(t) >= 2])}")

    return transactions

# =====================================================
# MODELO MELHORADO
# =====================================================
def train_improved_model(df_train):
    """
    Treina modelo com configurações otimizadas
    """
    print("\n🔧 Treinando modelo melhorado...")

    # Preparar transações (sem metadados)
    transactions = prepare_transactions_improved(df_train)

    # Converter para one-hot
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    transaction_df = pd.DataFrame(te_ary, columns=te.columns_)

    print(f"   • Dimensões: {transaction_df.shape}")
    print(f"   • Rotas únicas: {len([c for c in transaction_df.columns if '_to_' in c])}")

    # Treinar FP-Growth
    start = time.time()

    frequent_itemsets = fpgrowth(
        transaction_df,
        min_support=CONFIG_IMPROVED['MIN_SUPPORT'],
        use_colnames=True,
        max_len=CONFIG_IMPROVED['MAX_LEN']
    )

    print(f"   • {len(frequent_itemsets)} itemsets encontrados")

    # Gerar regras
    if len(frequent_itemsets) > 0:
        rules = association_rules(
            frequent_itemsets,
            metric="confidence",
            min_threshold=CONFIG_IMPROVED['MIN_CONFIDENCE']
        )

        # Filtrar apenas regras de rotas (não metadados)
        rules_filtered = rules[
            rules['antecedents'].apply(lambda x: all('_to_' in str(i) for i in x)) &
            rules['consequents'].apply(lambda x: all('_to_' in str(i) for i in x))
        ]

        # Aplicar filtro de lift
        rules_filtered = rules_filtered[rules_filtered['lift'] >= CONFIG_IMPROVED['MIN_LIFT']]

        # Ordenar e limitar
        rules_filtered = rules_filtered.nlargest(CONFIG_IMPROVED['TOP_RULES'], 'lift')

        exec_time = time.time() - start

        print(f"   • {len(rules_filtered)} regras de rotas válidas")
        print(f"   • Tempo: {exec_time:.2f}s")

        return rules_filtered, frequent_itemsets, transactions

    return pd.DataFrame(), frequent_itemsets, transactions

# =====================================================
# AVALIAÇÃO MELHORADA
# =====================================================
def evaluate_improved(rules, df_test):
    """
    Avaliação com método melhorado
    """
    print("\n📈 Avaliando modelo melhorado...")

    if len(rules) == 0:
        print("   ❌ Sem regras para avaliar")
        return {}

    # Preparar transações de teste
    test_transactions = prepare_transactions_improved(df_test)

    correct = 0
    total = 0
    coverage = 0

    # Criar dicionário de regras para busca rápida
    rules_dict = {}
    for _, rule in rules.iterrows():
        ant = tuple(rule['antecedents'])
        cons = tuple(rule['consequents'])
        if ant not in rules_dict:
            rules_dict[ant] = []
        rules_dict[ant].append({
            'consequent': cons,
            'confidence': rule['confidence'],
            'lift': rule['lift']
        })

    # Avaliar cada transação
    for transaction in test_transactions[:1000]:  # Limitar para performance
        if len(transaction) < 2:
            continue

        # Usar primeira metade como histórico
        split_point = len(transaction) // 2
        history = transaction[:split_point]
        actual = transaction[split_point:]

        # Fazer previsões
        predictions = []
        for i, item in enumerate(history):
            ant_tuple = (item,)
            if ant_tuple in rules_dict:
                for rule in rules_dict[ant_tuple]:
                    predictions.extend(rule['consequent'])

        if predictions:
            coverage += 1
            # Verificar se alguma previsão está correta
            if any(pred in actual for pred in predictions):
                correct += 1

        total += 1

    if total > 0:
        accuracy = correct / total
        coverage_rate = coverage / total

        print(f"   • Acurácia: {accuracy:.1%}")
        print(f"   • Cobertura: {coverage_rate:.1%}")
        print(f"   • Avaliados: {total}")

        return {
            'accuracy': accuracy,
            'coverage': coverage_rate,
            'total': total,
            'correct': correct
        }

    return {}

# =====================================================
# ANÁLISE DE PADRÕES
# =====================================================
def analyze_patterns(rules):
    """
    Analisa os padrões encontrados
    """
    print("\n🔍 Análise de Padrões:")
    print("="*70)

    # Rotas mais frequentes como antecedente
    all_antecedents = []
    for ant in rules['antecedents']:
        all_antecedents.extend(list(ant))

    from collections import Counter
    ant_counter = Counter(all_antecedents)

    print("\n📍 Top 5 Rotas de Origem (aparecem em regras):")
    for route, count in ant_counter.most_common(5):
        if '_to_' in route:
            origin, dest = route.split('_to_')
            print(f"   • {origin[:10]}... → {dest[:10]}... ({count} regras)")

    # Análise de lift
    print(f"\n📊 Distribuição de Lift:")
    print(f"   • Lift médio: {rules['lift'].mean():.2f}")
    print(f"   • Lift máximo: {rules['lift'].max():.2f}")
    print(f"   • Regras com lift > 10: {len(rules[rules['lift'] > 10])}")
    print(f"   • Regras com lift > 5: {len(rules[rules['lift'] > 5])}")

    # Análise de confiança
    print(f"\n📊 Distribuição de Confiança:")
    print(f"   • Confiança média: {rules['confidence'].mean():.1%}")
    print(f"   • Regras com confiança > 50%: {len(rules[rules['confidence'] > 0.5])}")
    print(f"   • Regras com confiança > 30%: {len(rules[rules['confidence'] > 0.3])}")

# =====================================================
# EXECUTAR ANÁLISE MELHORADA
# =====================================================
def run_improved_analysis():
    """
    Executa análise completa melhorada
    """
    # Carregar dados
    print("\n📂 Carregando dados...")
    df = pd.read_csv('df_curado.csv')

    # Usar amostra maior se disponível
    if len(df) > 100000:
        print(f"   • Dataset grande detectado: {len(df):,} registros")
        df = df.sample(n=100000, random_state=42)
        print(f"   • Usando amostra de 100k para performance")

    print(f"   • Registros: {len(df):,}")
    print(f"   • Clientes únicos: {df['client_id'].nunique():,}")

    # Dividir dados
    train_size = int(0.8 * len(df))
    df_train = df[:train_size]
    df_test = df[train_size:]

    print(f"   • Treino: {len(df_train):,}")
    print(f"   • Teste: {len(df_test):,}")

    # Treinar modelo melhorado
    rules, itemsets, transactions = train_improved_model(df_train)

    # Avaliar
    metrics = evaluate_improved(rules, df_test)

    # Analisar padrões
    if len(rules) > 0:
        analyze_patterns(rules)

    # Mostrar top regras
    print("\n🏆 TOP 10 REGRAS MELHORADAS:")
    print("="*70)

    for i, (_, rule) in enumerate(rules.head(10).iterrows(), 1):
        ant = list(rule['antecedents'])[0] if rule['antecedents'] else 'N/A'
        cons = list(rule['consequents'])[0] if rule['consequents'] else 'N/A'

        # Formatar rotas
        if '_to_' in ant and '_to_' in cons:
            ant_parts = ant.split('_to_')
            cons_parts = cons.split('_to_')

            print(f"\nRegra {i}:")
            print(f"  SE viaja: {ant_parts[0][:15]}... → {ant_parts[1][:15]}...")
            print(f"  ENTÃO: {cons_parts[0][:15]}... → {cons_parts[1][:15]}...")
            print(f"  Confiança: {rule['confidence']:.1%} | Lift: {rule['lift']:.2f}")

    return rules, metrics

# =====================================================
# EXECUTAR
# =====================================================
if __name__ == "__main__":
    rules, metrics = run_improved_analysis()

    print("\n" + "="*70)
    print("✅ ANÁLISE MELHORADA COMPLETA!")
    print("="*70)

    if metrics:
        improvement = metrics['accuracy'] - 0.083  # Comparando com 8.3% anterior
        if improvement > 0:
            print(f"🎯 Melhoria de acurácia: +{improvement:.1%}")

        print(f"""
        📊 Resumo Final:
        • Acurácia: {metrics['accuracy']:.1%}
        • Cobertura: {metrics['coverage']:.1%}
        • Regras úteis: {len(rules)}

        💡 Recomendações:
        1. Use sequências temporais (ordem importa)
        2. Adicione features de contexto (dia, hora, preço)
        3. Combine com modelo de ML supervisionado
        4. Use embeddings para rotas similares
        """)

🚀 MODELO MELHORADO - CLICKPREDICT

📂 Carregando dados...
   • Dataset grande detectado: 1,741,344 registros
   • Usando amostra de 100k para performance
   • Registros: 100,000
   • Clientes únicos: 78,659
   • Treino: 80,000
   • Teste: 20,000

🔧 Treinando modelo melhorado...

📊 Preparando transações (versão melhorada)...
   • 15174 transações válidas criadas
   • Transações com 2+ rotas: 15174
   • Dimensões: (15174, 6398)
   • Rotas únicas: 6398
   • 1004 itemsets encontrados
   • 500 regras de rotas válidas
   • Tempo: 10.35s

📈 Avaliando modelo melhorado...

📊 Preparando transações (versão melhorada)...
   • 2923 transações válidas criadas
   • Transações com 2+ rotas: 2923
   • Acurácia: 42.0%
   • Cobertura: 59.3%
   • Avaliados: 1000

🔍 Análise de Padrões:

📍 Top 5 Rotas de Origem (aparecem em regras):
   • fbb2a73b0b... → 7688b6ef52... (3 regras)
   • 6b86b273ff... → bbb965ab0c... (2 regras)
   • f0bc318fb8... → 6b86b273ff... (2 regras)
   • 7688b6ef52... → fbb2a73b0b... (2 re

In [15]:

# ====== 1) SALVAR O MODELO (pickle + resumo JSON) ======
import pickle, json
from datetime import datetime

def salvar_modelo_treinado(rules, metrics, caminho_modelo='modelo_fpgrowth.pkl', caminho_resumo='modelo_resumo.json'):
    print("\n" + "="*70)
    print("💾 SALVANDO MODELO TREINADO")
    print("="*70)

    modelo_completo = {
        'rules': rules,                  # DataFrame com colunas: antecedents, consequents, confidence, lift
        'metrics': metrics or {},        # dict: ex. {'accuracy': 0.23, 'coverage': 0.51}
        'config': {
            'MIN_SUPPORT': 0.0005,
            'MIN_CONFIDENCE': 0.2,
            'MIN_LIFT': 1.0,
            'MAX_LEN': 2,
            'TOP_RULES': 2000
        },
        'training_date': datetime.now().isoformat(),
        'version': '2.0'
    }

    with open(caminho_modelo, 'wb') as f:
        pickle.dump(modelo_completo, f)

    resumo = {
        'data_treino': modelo_completo['training_date'],
        'total_regras': int(len(rules)) if rules is not None else 0,
        'acuracia': float((metrics or {}).get('accuracy', 0.0)),
        'cobertura': float((metrics or {}).get('coverage', 0.0)),
    }
    with open(caminho_resumo, 'w') as f:
        json.dump(resumo, f, indent=2)

    print(f"✅ Modelo salvo em: {caminho_modelo}")
    print(f"✅ Resumo salvo em: {caminho_resumo}")
    print("\n📊 Estatísticas do modelo:")
    print(f"   • Regras: {resumo['total_regras']}")
    print(f"   • Acurácia: {resumo['acuracia']:.1%}")
    print(f"   • Cobertura: {resumo['cobertura']:.1%}")
    return modelo_completo

# 👉 Salva imediatamente (rules/metrics já devem existir no ambiente)
_ = salvar_modelo_treinado(rules, metrics)


# ====== 2) CARREGAR E USAR O MODELO PARA PREVER (CSV OU DF) ======
import os
import pandas as pd
from typing import List, Dict, Any

class PreditorRotas:
    """
    Preditor de rotas baseado em regras de associação (FP-Growth).
    O pickle deve conter a chave 'rules' (DataFrame) com colunas:
    ['antecedents','consequents','confidence','lift'].
    """
    def __init__(self, caminho_modelo: str = "modelo_fpgrowth.pkl"):
        print("📂 Carregando modelo...")
        if not os.path.exists(caminho_modelo):
            raise FileNotFoundError(f"Arquivo de modelo não encontrado: {caminho_modelo}")
        with open(caminho_modelo, "rb") as f:
            self.modelo = pickle.load(f)
        if "rules" not in self.modelo:
            raise KeyError("O pickle não contém a chave 'rules'.")
        self.rules = self.modelo["rules"]
        print(f"✅ {len(self.rules)} regras carregadas")

        # Índice: (antecedente,) -> lista de {dest, conf, lift}
        self.indice: Dict[Any, List[Dict[str, Any]]] = {}
        for _, rule in self.rules.iterrows():
            ant_list = self._to_list(rule["antecedents"])
            cons_list = self._to_list(rule["consequents"])
            ant_key = tuple(ant_list)
            self.indice.setdefault(ant_key, []).append({
                "dest": cons_list,
                "conf": float(rule["confidence"]),
                "lift": float(rule["lift"]),
            })

    @staticmethod
    def _to_list(x):
        if isinstance(x, (set, frozenset, tuple, list)):
            return list(x)
        return [x]

    def processar_csv(self, arquivo_csv: str, sep: str = ",", encoding: str = "utf-8", salvar: bool = True) -> pd.DataFrame:
        """
        Espera CSV com colunas:
          - client_id
          - origin_departure, destination_departure
          - origin_return, destination_return
          - no_return_flag  (True = sem volta / False = tem volta)
        """
        print(f"\n📊 Processando {arquivo_csv}...")
        if not os.path.exists(arquivo_csv):
            raise FileNotFoundError(f"CSV não encontrado: {arquivo_csv}")
        df = pd.read_csv(arquivo_csv, sep=sep, encoding=encoding)
        df_result = self.processar_dataframe(df)
        if salvar:
            saida = arquivo_csv.replace(".csv", "_previsoes.csv")
            df_result.to_csv(saida, index=False)
            print(f"✅ Salvo em {saida}")
        print("\n📋 Primeiras previsões:")
        print(df_result.head(10))
        return df_result

    def processar_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        colunas_esperadas = [
            "client_id",
            "origin_departure", "destination_departure",
            "origin_return", "destination_return",
            "no_return_flag",
        ]
        faltando = [c for c in colunas_esperadas if c not in df.columns]
        if faltando:
            raise KeyError(
                "Colunas faltando no DataFrame: "
                + ", ".join(faltando)
                + "\n💡 Ajuste seu CSV/DF para conter essas colunas."
            )

        resultados: List[Dict[str, Any]] = []
        for client_id in df["client_id"].dropna().unique():
            client_df = df[df["client_id"] == client_id]

            # Extrair rotas do cliente
            rotas: List[str] = []
            for _, row in client_df.iterrows():
                # ida
                if pd.notna(row["origin_departure"]) and pd.notna(row["destination_departure"]):
                    rotas.append(f"{row['origin_departure']}_to_{row['destination_departure']}")
                # volta (se houver)
                no_return = row.get("no_return_flag", True)
                if not bool(no_return):
                    if pd.notna(row.get("origin_return")) and pd.notna(row.get("destination_return")):
                        rotas.append(f"{row['origin_return']}_to_{row['destination_return']}")

            # Prever próximas rotas (top-5)
            previsoes = self.prever(rotas)
            for i, p in enumerate(previsoes[:5], 1):
                resultados.append({
                    "cliente": client_id,
                    "rank": i,
                    "rota": p["rota"],
                    "confianca": f"{p['conf']:.1%}",
                    "lift": f"{p['lift']:.1f}",
                })

        return pd.DataFrame(resultados)

    def prever(self, rotas_cliente: List[str]) -> List[Dict[str, Any]]:
        """
        Recebe lista de rotas já realizadas pelo cliente (strings 'A_to_B')
        e retorna lista única ordenada por (conf * lift).
        """
        previsoes: List[Dict[str, Any]] = []
        for rota in rotas_cliente:
            chave = (rota,)
            if chave in self.indice:
                for regra in self.indice[chave]:
                    for dest in regra["dest"]:
                        if dest not in rotas_cliente:
                            previsoes.append({
                                "rota": dest,
                                "conf": regra["conf"],
                                "lift": regra["lift"],
                            })

        vistos = set()
        unicos: List[Dict[str, Any]] = []
        for p in sorted(previsoes, key=lambda x: x["conf"] * x["lift"], reverse=True):
            if p["rota"] not in vistos:
                vistos.add(p["rota"])
                unicos.append(p)
        return unicos


# ====== 3) EXEMPLOS DE USO (ESCOLHA UM) ======
# (A) Usar um CSV já salvo em disco:
CSV_ENTRADA = "df_curado_head5000.csv"   # <-- troque pelo seu arquivo .csv
p = PreditorRotas("modelo_fpgrowth.pkl")
df_pred = p.processar_csv(CSV_ENTRADA)   # se seu CSV for ; e latin-1: p.processar_csv(CSV_ENTRADA, sep=";", encoding="latin-1")

# (B) OU, se você já tem um DataFrame em memória:
# df = pd.read_csv(CSV_ENTRADA)         # ou seu DF já carregado
# df_pred = p.processar_dataframe(df)

# Visualizar as primeiras linhas
df_pred.head(10)



💾 SALVANDO MODELO TREINADO
✅ Modelo salvo em: modelo_fpgrowth.pkl
✅ Resumo salvo em: modelo_resumo.json

📊 Estatísticas do modelo:
   • Regras: 500
   • Acurácia: 42.0%
   • Cobertura: 59.3%
📂 Carregando modelo...
✅ 500 regras carregadas

📊 Processando df_curado_head5000.csv...
✅ Salvo em df_curado_head5000_previsoes.csv

📋 Primeiras previsões:
                                             cliente  rank  \
0  3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...     1   
1  3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...     2   
2  ceea0de820a6379f2c4215bddaec66c33994b304607e56...     1   
3  e15109b0b8e9f6f1554e560837eb55543f035f91d8be4f...     1   
4  ec66b05934a8d00f661257ea91874241fccb4fb128028d...     1   
5  cf95d567cc38355ed310d70e5d43fb4af67a0373c7282d...     1   
6  2bda00506cf67ae2828006c8131bd215ada4943dd1738d...     1   
7  3265420405fa6251960db35d01bcdc0af0eda2dfaf5b96...     1   
8  3265420405fa6251960db35d01bcdc0af0eda2dfaf5b96...     2   
9  7c371df88bf42570a41524acf6d0d

,cliente,rank,rota,confianca,lift
0,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,1,8c1f1046219ddd216a023f792356ddf127fce372a72ec9...,83.3%,702.5
1,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,2,7688b6ef52555962d008fff894223582c484517cea7da4...,71.7%,14.2
2,ceea0de820a6379f2c4215bddaec66c33994b304607e56...,1,23765fc69c4e3c0b10f5d15471dc2245e2a19af16b513f...,63.0%,113.7
3,e15109b0b8e9f6f1554e560837eb55543f035f91d8be4f...,1,be47addbcb8f60566a3d7fd5a36f8195798e2848b36819...,65.0%,580.2
4,ec66b05934a8d00f661257ea91874241fccb4fb128028d...,1,7688b6ef52555962d008fff894223582c484517cea7da4...,61.1%,264.9
5,cf95d567cc38355ed310d70e5d43fb4af67a0373c7282d...,1,23765fc69c4e3c0b10f5d15471dc2245e2a19af16b513f...,63.0%,113.7
6,2bda00506cf67ae2828006c8131bd215ada4943dd1738d...,1,093434a3ee9e0a010bb2c2aae06c2614dd24894062a1ca...,48.7%,224.0
7,3265420405fa6251960db35d01bcdc0af0eda2dfaf5b96...,1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,61.9%,213.5
8,3265420405fa6251960db35d01bcdc0af0eda2dfaf5b96...,2,5c06e46c5e47cfacad16ce1e37f17c09fdbc7072c56761...,51.1%,198.7
9,7c371df88bf42570a41524acf6d0db9f4ec1f0631df16c...,1,5c06e46c5e47cfacad16ce1e37f17c09fdbc7072c56761...,51.1%,198.7
